<a href="https://colab.research.google.com/github/huangtinglin/test_colab/blob/main/CPSC483_colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GNN for link prediction and graph classification task

Last time we explore a standard benchmark datase Cora, and implement a classic graph neural network GCN(Kipf et al. (2017)) for node classification task. In this Colab, we are going to explore two kinds of graph learning task: **link prediction** and **graph classification**. We will apply GCN to the link prediction task, and implement a novel GNN model [GIN](https://arxiv.org/abs/1810.00826) for graph classification task. All of these implementations are still based on the [PyG](https://pytorch-geometric.readthedocs.io/en/latest/).


## Outline



- Link prediction task
- Graph classification task

In [1]:
# import the pytorch library into environment and check its version
import os
import torch
print("Using torch", torch.__version__)

Using torch 1.12.1+cu113


Let's start installing PyG by `pip`. The version of PyG should match the current version of PyTorch. Here we follow the [instruction](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html) of PyG:

In [2]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install ogb  # for datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 7.9 MB 7.9 MB/s 
     |████████████████████████████████| 3.5 MB 63.9 MB/s 
     |████████████████████████████████| 2.4 MB 70.9 MB/s 
     |████████████████████████████████| 709 kB 79.2 MB/s 
     |████████████████████████████████| 467 kB 8.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=f14f39f0929f79a8a67613d505ef276d6f7003197a7e07dd194ed39891c816c3
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 228 kB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.w

Import some required libraries into our environment:

In [10]:
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric import nn
import torch_geometric.transforms as T



## Link prediction task


### Dataset preprocess

As shown in the following figure, link prediction is to predict whether two nodes in a graph have a link, which can be considered as a binary classification task. We will construct a link prediction dataset containing training, validation, and test set based on Cora. 


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_example.png?raw=1" height="200" width="200"/>
</center>
<br/>


Given a graph, we divide the initial edge set into three distinct edge sets which represent the training, validation, and test set. Training set does not include edges in validation and test set, and the validation split does not include edges in the test split. Validation and test data should not be leaked into the training set.


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_dataset_split.png?raw=1" height="200" width="500"/>
</center>
<br/>


Our model will be optimized on the training set. We can use `transforms` function in PyG to easily generate the data splits:

In [75]:
transform = T.Compose([
    T.RandomLinkSplit(num_val=0.1,  # ratio of edges including in the validation set
                      num_test=0.2,  # ratio of edges including in the test set
                      is_undirected=True,
                      add_negative_train_samples=False),
])

Loading the Cora dataset:

In [76]:
dataset = Planetoid('/tmp/cora', 'cora', transform=transform)

The data will be transformed from a data object to three tuples, where each element represents the corresponding split:

In [77]:
train_data, val_data, test_data = dataset[0]

Printing the statistics of data:

In [80]:
print("Number of the nodes in training, validation and test data are", train_data.num_nodes, val_data.num_nodes, test_data.num_nodes)
print("Number of the edges in training, validation and test data are", train_data.num_edges, val_data.num_edges, test_data.num_edges)

Number of the nodes in training, validation and test data are 2708 2708 2708
Number of the edges in training, validation and test data are 7392 7392 8446


### Model Building

We constructed the GCN by PyG in the last Colab, and now we simply use the same architecture:

In [93]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, node_feature, edge_index):

        output = self.conv1(node_feature, edge_index)
        output = self.act(output)
        output = self.conv2(output, edge_index)

        return output

Initializing a GCN model:

In [94]:
model = GCN(dataset.num_features, hidden_channels=128, out_channels=64)

Similar as the what we do in the node classification task, we first apply the GCN model to produce the representation of each node in the graph. Usually we will use **inner product** to measure the similarity between two node representations to determine how likely it is for these two nodes to be connected.

#### Question 1

Following the instruction and implement the function to calculate the inner product:

In [88]:
def compute_similarity(node_embs, edge_index):
    result = 0

    # TODO: Define similarity function.
    # 1. calculate the inner product between all the pairs in the edge_index
    # Note: the shape of node_embs is [n, h] where n is the number of nodes, and h is the embedding size
    # the shape of edge_index is [2, m] where m is the number of edges

    ############# Your code here ############
    ## (~1 line of code)
    result = (node_embs[edge_index[0]] * node_embs[edge_index[1]]).sum(dim=-1)

    #########################################

    return result

n, h = 5, 10  # number of nodes and embedding size
node_embs = torch.rand(n, h)
edge_index = torch.tensor([[0, 1, 2, 3], 
                           [2, 3, 0, 1]])  # compute the similarity of (0, 2), (1, 3), (2, 0), (3, 1)
similarity = compute_similarity(node_embs, edge_index)
print("Similairty:", similarity)

Similairty: tensor([3.2714, 2.8681, 3.2714, 2.8681])


We optimize the model by minimizing the loss function. Here we consider the link prediction task as a binary classification task (edge exists or no), and apply binary cross entropy loss:

In [89]:
criterion = torch.nn.BCEWithLogitsLoss()

In [14]:
origin_dataset = Planetoid('/tmp/cora', 'cora')

In [30]:
origin_dataset[0].num_edges

10556

In [81]:
origin_dataset[0].edge_index.shape

torch.Size([2, 10556])

In [50]:
dataset = Planetoid('/tmp/cora', 'cora', transform=transform)

In [51]:
train_data, val_data, test_data = dataset[0]

In [52]:
train_data.edge_index

tensor([[1740, 1957, 1225,  ..., 2432, 2321,  760],
        [2391, 2241, 2579,  ..., 2431, 2266,  567]])

In [53]:
val_data.edge_index

tensor([[1740, 1957, 1225,  ..., 2432, 2321,  760],
        [2391, 2241, 2579,  ..., 2431, 2266,  567]])

In [65]:
test_data.num_edges

8446

In [61]:
edge_index = origin_dataset[0].edge_index

In [62]:
mask = edge_index[0] <= edge_index[1]
perm = mask.nonzero(as_tuple=False).view(-1)
perm = perm[torch.randperm(perm.size(0), device=perm.device)]

In [64]:
perm.shape

torch.Size([5278])

In [67]:
num_val = int(0.2 * perm.numel())
num_val

1055

In [70]:
num_test = int(0.2 * perm.numel())
num_test

1055

In [71]:
num_train = perm.numel() - num_val - num_test
num_train

3168

In [72]:
train_edges = perm[:num_train]
val_edges = perm[num_train:num_train + num_val]
test_edges = perm[num_train + num_val:]
train_val_edges = perm[:num_train + num_val]

In [74]:
val_edges.shape

torch.Size([1055])

Firstly, we load the Cora dataset:

In [4]:
dataset = Planetoid('/tmp/cora', 'cora')
data = dataset[0]

Processing...
Done!


In [ ]:
# import torch_geometric.data into environment
from torch_geometric.data import Data

We have 6 edges (undirected graph) and 3 nodes in this graph. So the edge index can be defined as:

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2, 0, 2],
                           [1, 0, 2, 1, 2, 0]], dtype=torch.long)

Each edge is represented as a tuple (u, v), and that edge_index consists of num_edges columns where each column consists of the two indices u and v corresponding to each edge.

Besides, each node can have a node feature which describes the node's property:

In [ ]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

Then we can define a `Data` object with edge index and node attribute:

In [ ]:
data = Data(x=x, edge_index=edge_index)

`Data` object supports many useful utility functions. For example, we can see the number of the nodes, and whether the graph is a undirected graph:

In [ ]:
num_nodes = data.num_nodes
print("number of nodes is:", num_nodes)

is_directed = data.is_directed()
print("graph is directed or not:", is_directed)

### Question 1 (5 points)

What is the number of the neighbors of node 0 in the graph?

In [ ]:
def get_n_neighbors(graph, idx):
  # TODO: Implement a function that takes a Data object,
  # an index of a node, and returns the number of the neighbors 
  # of this node (as an integer).

  n_neighbors = 0

  ############# Your code here ############
  ## (~1 line of code)

  #########################################

  return n_neighbors

idx = 0
n_neighbors = get_n_neighbors(data, idx)
print('Node with index {} has {} neighbors'.format(idx, n_neighbors))

PyG has a number of graph data with various scales. Cora is one of the most famous dataset in graph learning, and we can use it by PyG:

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid('/tmp/cora', 'cora')
data = dataset[0]

We can see the number of the nodes and edges in cora:

In [ ]:
num_nodes = data.num_nodes
print('cora has {} nodes'.format(num_nodes))

num_edges = data.num_edges
print('cora has {} edges'.format(num_edges))

### Question 2 (10 points)

1. What is the number of the classes in cora dataset? 
2. Which node in Cora has the most number of neighbors?

In [ ]:
def get_num_classes(data):
  # TODO: Implement a function that takes a dataset object
  # and returns the number of classes for that dataset.

  num_classes = 0

  ############# Your code here ############
  ## (~1 line of code)
  
  #########################################

  return num_classes

def get_idx_with_most_neighbors(data):
  # TODO: Implement a function that takes a dataset object
  # and returns the node's index that has the greatest amount of neighbors.

  idx = -1

  ############# Your code here ############
  ## (~3 line of code)
  
  #########################################

  return idx

num_classes = get_num_classes(data)
print("cora has {} classes".format(num_classes))

idx = get_idx_with_most_neighbors(data)
print("{} in cora has the most number of neighbors".format(idx))

In cora, we split the data into train set, validation set and test set by node mask. All the nodes will participate in the message passing process, but we can only assess the train label during training process. This is what we call [transductive learning](https://en.wikipedia.org/wiki/Transduction_(machine_learning)).

In [ ]:
node_feature = data.x

train_node_feature = node_feature[data.train_mask]
valid_node_feature = node_feature[data.val_mask]
test_node_feature = node_feature[data.test_mask]

print("number of nodes in train set,", train_node_feature.shape[0])
print("number of nodes in valid set,", valid_node_feature.shape[0])
print("number of nodes in test set,", test_node_feature.shape[0])

## Build a GNN by PyG

In this section we will use PyG to build a classic graph neural network called GCN([Kipf et al. (2017)](https://arxiv.org/pdf/1609.02907.pdf)). Then we will apply this model to handle node classification task in cora.
A GCN is built by stacking multiple graph convolution layers `GCNConv` which passes the messages from neighbors to the center node. Here we can define a `GCNConv` by PyG:


In [ ]:
from torch_geometric.nn import GCNConv

conv = GCNConv(in_channels=1433, out_channels=200, normalize=True)

`in_channels` is the dimension of node's input feature, `out_channels` is the  dimension of the output representation of node, and `normalize` is whether to add self-loops and compute symmetric normalization on the adjacent matrix. 
The feature's dimension in cora is 1433, so `in_channels` is set as 1433. We can perform a message passing on cora like this:

In [ ]:
node_feature = data.x
edge_index = data.edge_index

node_representation = conv(node_feature, edge_index)

print("dimension of node_feature:", node_feature.shape)
print("dimension of node_representation:", node_representation.shape)

We can see that the inputs of `GCNConv` are node feature and edge index. Then the convolution module will perform a message passing like GCN. 
Recall the MLP we build in colab0. Here we also use `nn.Module` to define a MLP class containing the basic modules of GCN. 

### Question 3 (10 points)

Following the instruction and build a GCN class using the `GCNConv` modules. 


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        # TODO: Define two GCNConv modules and a ReLU function.
        # The input size and output size of first GCNConv module should be in_channels and hidden_channels
        # The input size and output size of second GCNConv module should be hidden_channels and out_channels

        ############# Your code here ############
        ## (~3 line of code)

        #########################################

    def forward(self, node_feature, edge_index):

        output = None

        # TODO: Use the modules you define in __init__ to perform message passing.
        # ReLU function should be used in the middle of two GCNConv modules.

        ############# Your code here ############
        ## (~3 line of code)

        #########################################

        return output

## Training and Testing

Now we can try to construct training and testing pipeline, which is similar to what we do in colab0. First we initialize a GCN model:

In [ ]:
hidden_channels = 200
num_features = dataset.num_features
num_classes = 0  # please write down the number of classes

model = GCN(num_features, hidden_channels, num_classes)

Then we define the optimizer and loss function. Since it is a classification task, we use Cross Entropy Loss:

In [ ]:
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

### Question 4 (10 points)

Please follow the instruction and implement a function that trains a model. 

In [ ]:
def train(model, data, optimizer, loss_fn):

    loss = 0

    # TODO: Define train function.
    # 1. put the model into train mode
    # 2. clear the gradients calculated from the last batch
    # 3. get the prediction by model
    # 4. calculate the loss between our predictions and the actual labels. 
    # Just using nodes in train set!
    # 5. calculate the gradients of each parameter
    # 6. update the parameters by taking an optimizer step

    ############# Your code here ############
    ## (~7 line of code)

    #########################################

    return loss

### Question 5 (10 points)

Please follow the instruction and implement a function that evaluates a model in train, valid and test sets.

In [ ]:
@torch.no_grad()
def test(model, data):

    accuracy_list = [0, 0, 0]

    # TODO: Define test function.
    # 1. put the model into eval mode
    # 2. get the prediction by model
    # 3. calculate the accuracy for each set
    # NOTE: the results should be a list containing the accuracy of different set

    ############# Your code here ############
    ## (~5 line of code)

    #########################################

    return accuracy_list

We can start to train our model with `train` and `test` functions:

In [ ]:
epochs = 10

best_val_acc = final_test_acc = 0
for epoch in range(1, epochs + 1):
    loss = train(model, data, optimizer, loss_fn)
    train_acc, val_acc, test_acc = test(model, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        final_test_acc = test_acc
print("after {} epochs' training, the best test accuracy is {}".format(epochs, final_test_acc))

after 10 epochs' training, the best test accuracy is 0


## Submission

Make sure to run all the cells and save a copy of this colab in your driver. If you complete this notebook, download the colab and upload your work to canvas to submit it.